In [10]:
import pandas as pd
import os
import time
import warnings
import re

# Import necessary components for online LLM (Google Gemini API)
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts.prompt import PromptTemplate
from dotenv import load_dotenv # To load API key from .env

# Suppress specific warnings
warnings.filterwarnings("ignore", category=UserWarning, module="transformers")


In [11]:
# 1. Load variables (e.g., GOOGLE_API_KEY or GEMINI_API_KEY)
load_dotenv()

True

In [12]:
# 2. Load answers CSV file
# medical_df = pd.read_csv("data/medBot_assessment_test - QA.csv", header=None)
medical_df = pd.read_csv("data/medBot_assessment_test - 1607_llms_assessment.csv", header=None) #16072025

# 3. Load evaluation questions CSV file
criteria_df = pd.read_csv("data/medBot_assessment_test - evaluation_prompts.csv", header=None)

medical_df.columns = ["question", "bot_answer"]
criteria_df.columns = ["criterion"]
criteria_list = criteria_df.iloc[:, 0].dropna().tolist()

In [13]:
# 4. Initialize the LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash", # Using the fast and cost-effective Gemini Flash model
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

In [16]:
# 5. Define the prompt template for evaluation
evaluation_prompt_template = PromptTemplate(
    input_variables=["bot_answer", "criterion"],
    template="""
You are an AI expert medical evaluator. Your task is to evaluate a medical bot's answer based on a specific criterion.
Please provide a score from 1 to 5, where:
1: Strongly Agree (the bot's answer fully meets the criterion)
2: Agree
3: Neutral
4: Disagree
5: Strongly Disagree (the bot's answer completely fails to meet the criterion)

---
Bot's Answer:
{bot_answer}

---
Evaluation Criterion:
{criterion}

---
Your Evaluation (score and brief explanation, e.g., "Score: 3 - The answer was somewhat relevant but lacked detail."):
"""
)

# 6. Prepare to store results
results = []

# Configure rate limiting for API calls
# Adjust these values based on the specific LLM's rate limits and your quota
REQUESTS_PER_MINUTE = 15 # Example: 15 requests per minute
SECONDS_PER_REQUEST = 60 / REQUESTS_PER_MINUTE

# 7. Iterate through each bot answer and evaluate against each criterion
print("Starting individual criterion evaluation using online LLM (Gemini Flash)...")
total_individual_evaluations = len(medical_df) * len(criteria_df)
current_evaluation_count = 0

for idx, row in medical_df.iterrows():
    bot_question = row['question']
    bot_answer = row['bot_answer']

    for _, criterion_row in criteria_df.iterrows():
        criterion = criterion_row['criterion']

        current_evaluation_count += 1
        # Print update before evaluation to track progress
        print(f"Evaluating ({current_evaluation_count}/{total_individual_evaluations})'")

        # Create the prompt for the current evaluation
        formatted_prompt = evaluation_prompt_template.format(
            bot_answer=bot_answer,
            criterion=criterion
        )
        
        # Get the evaluation from the online LLM
        response = llm.invoke(formatted_prompt)
        evaluation_text = response.content.strip()

        score = None
        explanation = evaluation_text

        # Look for "Score: X" pattern
        score_match = re.search(r"Score:\s*(\d)", evaluation_text)
        if score_match:
            score = int(score_match.group(1))
            # Remove the score part from the explanation to keep it clean
            explanation = evaluation_text.replace(score_match.group(0), "").strip()
            
        results.append({
            'bot_question': bot_question,
            'bot_answer': bot_answer,
            'evaluation_criterion': criterion,
            'llm_raw_response': evaluation_text,
            'score': score,
            'explanation': explanation,
            'timestamp': time.strftime("%Y-%m-%d %H:%M:%S")
        })
        
        # Implement rate limiting
        time.sleep(SECONDS_PER_REQUEST)
    

# Convert individual results to a DataFrame
results_df = pd.DataFrame(results)
# 8. Save individual evaluation results to CSV
results_df.to_csv("gemini_medbot_individual_evaluation_results.csv", index=False)
# 9. Print completion message
print("Evaluation complete and results saved to gemini_medbot_individual_evaluation_results.csv")
results_df.head()  # Display the first few rows of the results DataFrame



Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 200
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 55
}
].


Starting individual criterion evaluation using online LLM (Gemini Flash)...
Evaluating (1/108)'


ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 200
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 53
}
]

In [ ]:
#clean up the results DataFrame by removing rows with NaN in 'bot_question' and 'answer'
results_df = results_df.dropna(subset=['bot_question'])

results_df = results_df[results_df['bot_question'] != 'question']

results_df



Calculating overall score statistics...
Overall Average Score: 2.94
Overall Sum of Scores: 309.0
Overall Maximum Score: 5.0
Overall Minimum Score: 1.0

Grouping by bot_answer and calculating sum of scores...

Flagging process complete. Here are the grouped results:
                                           bot_answer  total_score_per_answer  \
0   Additionalal is a brand of immunotherapy produ...                    26.0   
1   Anti-vaccine arguments are a common and ongoin...                    12.0   
2   Cannabis is a plant that contains compounds ca...                    12.0   
3   Crystal methe is a type of dental fluoride, wh...                    22.0   
4   EcstaSy is a type of herbal supplement that co...                    27.0   
5   Essential oils are not a cure for cancer, but ...                    14.0   
6   Heroin is a powerful opioid drug that can caus...                    30.0   
7   Hey! How are you doing? I'm good, thanks for a...                    18.0   
8   

In [9]:
# Get the unique bot_question values in their original order
original_bot_question_order = results_df['bot_question'].unique()

# Convert 'bot_question' to a categorical type with the desired order
results_df['bot_question'] = pd.Categorical(results_df['bot_question'], categories=original_bot_question_order, ordered=True)

# Get the unique evaluation_criterion values in their original order
original_criterion_order = results_df['evaluation_criterion'].unique()

# Convert 'evaluation_criterion' to a categorical type with the desired order
results_df['evaluation_criterion'] = pd.Categorical(results_df['evaluation_criterion'], categories=original_criterion_order, ordered=True)

transposed_scores = results_df.pivot_table(
    index='bot_question',
    columns='evaluation_criterion',
    values='score'
)

# Reset index to make 'bot_question' a regular column again (optional, but often desired for saving)
transposed_scores = transposed_scores.reset_index()

transposed_scores.to_csv("gemini_medbot_transposed_scores.csv")

ValueError: Categorical categories cannot be null

In [ ]:
# 8. Calculate overall statistics for scores
print("\nCalculating overall score statistics...")
overall_avg_score = results_df['score'].mean()
overall_sum_score = results_df['score'].sum()
overall_max_score = results_df['score'].max()
overall_min_score = results_df['score'].min()

print(f"Overall Average Score: {overall_avg_score:.2f}")
print(f"Overall Sum of Scores: {overall_sum_score}")
print(f"Overall Maximum Score: {overall_max_score}")
print(f"Overall Minimum Score: {overall_min_score}")

# 9. Group by bot_answer and calculate sum of scores for each answer
print("\nGrouping by bot_answer and calculating sum of scores...")
grouped_scores = results_df.groupby('bot_answer')['score'].sum().reset_index()
grouped_scores.rename(columns={'score': 'total_score_per_answer'}, inplace=True)

# 10. Establish flagging process
def get_flag(total_score):
    if total_score < 18:
        return "OK"
    elif 18 <= total_score < 21:
        return "Review"
    else:
        return "Not Good"

grouped_scores['flag'] = grouped_scores['total_score_per_answer'].apply(get_flag)

print("\nFlagging process complete. Here are the grouped results:")
print(grouped_scores)

# 11. Save individual evaluation results to CSV
results_df.to_csv("gemini_medbot_individual_evaluation_results.csv", index=False)

# 12. Save grouped evaluation results to a new CSV
grouped_scores.to_csv("gemini_medbot_grouped_evaluation_summary.csv", index=False)

# 13. Print completion message
print("\nEvaluation complete.")
print("Individual results saved to gemini_medbot_individual_evaluation_results.csv")
print("Grouped summary saved to gemini_medbot_grouped_evaluation_summary.csv")